## Data Downloader

This Jupyter notebook downloads data about globally top selling games from digital gaming platform Steam. These data contain title, release date and information about reviews and prices of individual games. Output of this Downloader is CSV file.

First, we download a few packages necessary for our downloader to be able to scrape data from Steam webpage, then other packages help us display raw data and pandas help us to create the dataframe.

In [1]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display, HTML
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

We begin by creating a class, which we call 'Downloader'. Functions of few first attributes of this downloader come naturally from their names. Then we create attributes 'dataf', 'hoarder' and 'download_data'. As the list of games is too long for just one webpage, Steam saved it in more than 600 webpages in total. Thus, downloading our data works in following way, using  'download_data' attribute:
- It begins by creating a list of urls of given number of pages (starting from the first one) using 'hoarder' attribute.
- Then it uses 'dataf' attribute, which downloads HTML for every url in the list, applies first few attributes of class 'Downloader' and creates pandas dataframe of data for every given url.
- Last, 'download_data' appends dataframe for every url to one large dataframe we want to end up with. It also assigns indexes to individual games.

In [106]:
#number of pages to scrape

pages = 1

class Downloader:
    def __init__(self, link):
        self.link = link
        self.uClient = uReq(link)
        self.page_html = self.uClient.read()
        self.uClient.close()
        self.soup = BeautifulSoup(self.page_html, "lxml")
    
    def get_titles(self):
        td = self.soup.findAll('span', {"class":"title"})
        titles = []
        for ind in td:
            # lstrip and rstrip remove symbols from sides, strip removes white spaces
            titles.append(str(ind).lstrip('<span class="title">').rstrip('span>').rstrip('</'))
        return titles
    
    def get_release_dates(self):
        td = self.soup.findAll('div', {"class":"col search_released responsive_secondrow"})
        release_dates = []
        for ind in td:
            if ind.text != "":
            # lstrip and rstrip remove symbols from sides, strip removes white spaces
                release_dates.append(str(ind).rstrip('</div>').split(">")[-1])
            else:
                release_dates.append(None)
        return release_dates
    
    def reviews(self):
   
        td = self.soup.findAll('span', {"class":"search_review_summary mixed"} or
                               {"class":"search_review_summary positive"} or
                              {"class":"search_review_summary negative"})
        reviews = []
        for ind in td:
            # lstrip and rstrip remove symbols from sides, strip removes white spaces
            reviews.append(str(ind).split("html=")[-1])
        return reviews
     
    def get_share_positive_reviews(self):
        text = self.reviews()
        shares = []
        for percent in text:
            shares.append(percent.split("%")[0].split("br&gt;")[1])
        return shares
        
    def get_number_user_reviews(self):  
        text = self.reviews()
        numbers = []
        for number in text:
            start = number.find("of the ") + len("of the ")
            end = number.find(" user reviews")
            numbers.append(number[start:end].replace(",",""))
        return numbers
        
    def get_prices(self):

        td = self.soup.findAll('div', {"class":"col search_price_discount_combined responsive_secondrow"} or 
                          {"class":"col search_price discounted responsive_secondrow"})
        prices = []
        for ind in td:
            if "888888" not in str(ind):
                if (len(str(ind).split("\r\n")[-1].split("</div>\n</div>")[0].strip()) <10):
                    prices.append(str(ind).split("\r\n")[-1].split("</div>\n</div>")[0].strip().replace("€","").replace(",",".").replace("-","0").replace("Free","0"))
            
                else:
                    prices.append(None)
            else:
                start1 = str(ind).find("><strike>") + len("><strike>")
                end1 = str(ind).find("</strike>")
                prices.append(str(ind)[start1:end1].replace("€","").replace(",",".").replace("-","0").replace("Free","0"))
        return prices
    
    def get_price_after_sale(self):

        td = self.soup.findAll('div', {"class":"col search_price_discount_combined responsive_secondrow"} or 
                          {"class":"col search_price discounted responsive_secondrow"})
        sales = []
        for ind in td:
            if "888888" not in str(ind):
                sales.append(None)
            #elif (len(str(ind).split("\r\n")[-1].split("</div>\n</div>")[0].strip()) >10):
                #sales.append(0)
            else:
                sales.append(str(ind).split("br/>")[-1].split("€")[0].replace(",",".").replace("-","0").replace("Free","0").strip())
        return sales
    
    def get_rate_of_sale(self):

        td = self.soup.findAll('div', {"class":"col search_price_discount_combined responsive_secondrow"} or 
                          {"class":"col search_price discounted responsive_secondrow"})
        percent = []
        for ind in td:
            if "888888" not in str(ind):
                percent.append(None)
            else:
                start = str(ind).find(">\n<span>-")+len(">\n<span>-")
                end = str(ind).find("%")
                percent.append(str(ind)[start:end])
        return percent
    
    def dataf(self):
        titles = self.get_titles()
        dates = self.get_release_dates()
        share_reviews = self.get_share_positive_reviews()
        number_reviews = self.get_number_user_reviews()
        normal_prices = self.get_prices()
        sale_price = self.get_price_after_sale()
        sale_rate = self.get_rate_of_sale()
        
        
        self.data = pd.DataFrame({
             'Title': pd.Series(titles),
             'Release date': pd.to_datetime(pd.Series(dates),format='%d %b, %Y', errors = 'coerce'),
             'Share of positive reviews (in %)': pd.to_numeric(share_reviews, errors = 'coerce'),
             'Total number of reviews': pd.to_numeric(number_reviews, errors = 'coerce'),
             'Normal price (€)': pd.to_numeric(normal_prices, errors = 'coerce'),
             'Discounted price if there is a sale (€)': pd.to_numeric(sale_price, errors = 'coerce'),
             'Sale rate (in %)': pd.to_numeric(sale_rate, errors = 'raise')})
        return self.data
   
    def hoarder(self):
        urls = []
        #last_page = int(self.soup.findAll('a',{'class':"search_result_row ds_collapse_flag  app_impression_tracked"})[-1].find('a')['href'][-2:])
        for i in range(pages):
            urls.append(self.link + f"&page={451+i}")
        return urls
    
    def download_data(self):
        urls = self.hoarder()
        Frame = pd.DataFrame()
        for url in urls:
            print(url)
            Frame = Frame.append(pd.DataFrame(data = Downloader(url).dataf()))
        Frame.index = range(pages*25)
        return Frame
    
    
    
        #451+i
        #df = pd.DataFrame.from_dict(d, orient='index')
        #df.transpose()

This is the link for Steam page of global top sellers ordered by reviews.

In [107]:
link = 'https://store.steampowered.com/search/?sort_by=Reviews_DESC&os=win&filter=globaltopsellers'
link2 = 'https://store.steampowered.com/search/?sort_by=Reviews_DESC&os=win&filter=globaltopsellers&page=451'
second = Downloader(link2)
uClient = uReq(link2)
page_html = uClient.read()
uClient.close()
soup = BeautifulSoup(page_html, "lxml")
print(len(second.get_rate_of_sale()))
print(len(second.get_titles()))
print(len(second.get_release_dates()))
print(len(second.get_share_positive_reviews()))
print(len(second.get_number_user_reviews()))
print(len(second.get_prices()))
print(len(second.get_price_after_sale()))

25
25
25
20
20
25
25


# The Data

First, we initialize the link in order to continue working with it. Next, we can apply the 'download_data' attribute and take a look at the craped data.

In [85]:
first = Downloader(link)
#first.download() # in order to explore page html in a reasonable way, one can use online javascript beautifier, available at:
# beautifier.io
df = first.download_data()
df

https://store.steampowered.com/search/?sort_by=Reviews_DESC&os=win&filter=globaltopsellers&page=451


ValueError: arrays must all be same length

# Saving the data as CSV file

Now we can save the data as CSV file.

In [9]:
df.to_csv('steam_global_sellers_by_reviews')